# <b>CONVERSOR DE NOMES LÓGICOS PARA FÍSICOS</b>

## <b>1. Instalação e importação de bibliotecas</b>

In [1]:
# Descomente a linha a seguir apenas no primeiro uso. A lib "openpyxl" é necessária para o Pandas ler arquivos do Excel
#!pip install openpyxl

In [2]:
import pandas as pd
from unidecode import unidecode
from typing import Union, List
import re
from itertools import combinations

## <b>2. Importação do Glossário de termos</b>

In [3]:
# Abre o glossário no formato Excel (altere a localização do glossário, se necessário)
df_glossario = pd.read_excel('Glossário de termos.xlsx')

## <b>3. Função de conversão de nome lógico</b>

In [4]:
def converter_para_nome_fisico(glossario: pd.DataFrame, nomes_logicos: Union[str, List[str]]) -> pd.DataFrame:
    # Conjunto de palavras a serem removidas do nome lógico
    PALAVRAS_REMOVER = {'o', 'as', 'os', 'um', 'uma', 'uns', 'umas', 'de', 'do', 'da', 'dos', 'das', 'em', 'no', 'na', 'nos', 'nas', 'por', 'para', 'com', 'sem', 'sobre', 'entre', 'e', 'ou', 'mas', 'se', 'que'}
    
    # Dicionário de substituições baseado no glossário
    substituicoes = dict(zip(glossario['NOME'].str.strip(), glossario['ABREVIAÇÃO'].str.strip()))

    def construir_expressoes(palavras: List[str]) -> List[str]:
        expressoes = []
        for i in range(1, len(palavras) + 1):
            for comb in combinations(palavras, i):
                expressoes.append(' '.join(comb))
        return expressoes

    def converter(nome_logico: str) -> str:
        nome_logico_normalizado = unidecode(nome_logico.strip().upper())
        
        palavras = [p for p in nome_logico_normalizado.split()]
        
        expressoes_possiveis = construir_expressoes(palavras)
        expressoes_possiveis = sorted(expressoes_possiveis, key=len, reverse=True)

        for expressao in expressoes_possiveis:
            if expressao in substituicoes:
                #nome_logico_normalizado = re.sub(r'\b' + re.escape(expressao) + r'\b', substituicoes[expressao], nome_logico_normalizado)
                padrao = re.compile(re.escape(expressao))
                nome_logico_normalizado = padrao.sub(substituicoes[expressao], nome_logico_normalizado)

        contador = 0
        for n in nome_logico_normalizado.split():
            if n in palavras and n.lower() not in PALAVRAS_REMOVER:
                contador += 1

        if contador > 0:
            return f'{contador} palavra{'s' if contador > 1 else ''} não fo{'ram' if contador > 1 else 'i'} identificada{'s' if contador > 1 else ''} no glossário'
        
        return '_'.join(t for t in nome_logico_normalizado.split() if t.lower() not in PALAVRAS_REMOVER)

    # Garante que nomes_logicos seja uma lista
    nomes_logicos = [nomes_logicos] if isinstance(nomes_logicos, str) else nomes_logicos
    
    # Aplica a conversão para cada nome lógico
    nomes_fisicos = [converter(nome) for nome in nomes_logicos]

    # Retorna um DataFrame com os resultados
    return pd.DataFrame({'Nome Lógico': nomes_logicos, 'Nome Físico': nomes_fisicos})

## <b>4. Lista de nomes lógicos a serem convertidos</b>

In [5]:
nomes_logicos = [
    'A granel',
    'Ponta "B"',
    'São Paulo Futebol Clube',
    'Agência Nacional de Saúde Suplementar',
    'Agência-Tronco',
    'Alívio de numerário',
    'AnALiSe De SeNsIbIlIdAdE',
    'Assistência técnica',
    'Código do Curso',
    'Nome do Curso',
    'Texto do Curso',
    'Quantidade de Carga Horária do Curso',
    'Carga Funcionário Horária do Curso'
]

## <b>5. Conversão para nomes físicos</b>

In [6]:
nomes_fisicos = converter_para_nome_fisico(df_glossario, nomes_logicos)

In [7]:
nomes_fisicos

,Nome Lógico,Nome Físico
0,A granel,GRAN
1,"Ponta ""B""",PNTB
2,São Paulo Futebol Clube,3 palavras não foram identificadas no glossário
3,Agência Nacional de Saúde Suplementar,ANS
4,Agência-Tronco,AGTC
5,Alívio de numerário,ALNM
6,AnALiSe De SeNsIbIlIdAdE,ANLS
7,Assistência técnica,ATCN
8,Código do Curso,CD_CSO
9,Nome do Curso,NM_CSO
